In [1]:
#! wget -r -e robots=off -P ../ --no-parent  --no-host-directories --reject="index.html*" --convert-links http://tablebase.lichess.ovh/tables/standard/3-4-5/

In [2]:
import chess
from chess import syzygy
from board.BoardEncoder import BoardEncoder
import numpy as np
from montecarlo.MonteCarloNode import MonteCarloNode
import pickle
import os
from datetime import datetime
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [3]:
tablebase = syzygy.Tablebase()
tablebase.add_directory('../tables/standard/3-4-5')

290

## Probing tablebase for selected positions

We are using Syzygy tablebases. More information here: https://python-chess.readthedocs.io/en/latest/syzygy.html

In [4]:
def print_dtz(fen):
    dtz = tablebase.probe_dtz(chess.Board(fen))
    print(f"Distance to checkmate or a zeroing move: {dtz}")


White to move:

![](https://fen2image.chessvision.ai/8/8/8/8/3k4/8/3K4/3Q4_w_-_-_0_1?color=white)


In [5]:
print_dtz('8/8/8/8/3k4/8/3K4/3Q4 w - - 0 1')

Distance to checkmate or a zeroing move: 13


A positive value for DTZ indicates that the side to move needs 13 moves to win (or zero the 50-move counter), so here white is winning.


White to move:

![](https://fen2image.chessvision.ai/k7/p7/8/K7/8/8/8/8_w_-_-_0_1?color=white)


In [6]:
print_dtz('k7/p7/8/K7/8/8/8/8 w - - 0 1')

Distance to checkmate or a zeroing move: 0


Zero means that the position is drawn.

White to move:

![](https://fen2image.chessvision.ai/8/R7/3r4/8/4K3/8/4p3/4k3%20w%20-%20-%200%201?color=white)

In [7]:
print_dtz('8/R7/3r4/8/4K3/8/4p3/4k3 w - - 0 1')

Distance to checkmate or a zeroing move: -10


A negative value indicates that the side to play will lose in 10 moves, assuming the best play by both sides.

In [8]:
def dtz_to_eval(dtz):
    if dtz > 0:
        return max(101 - dtz, 50)
    elif dtz < 0:
        return min(-101-dtz, -50)
    return 0

## Building a training dataset

In [9]:
def bitarray_to_ndarray(bitarray):
    return np.array([int(bit) for bit in bitarray], dtype=bool)

def put_piece(board, color, piece, check_validity=True):
    while True:
        square = np.random.randint(0, 64)
        while board.piece_at(square) is not None:
            square = np.random.randint(0, 64)
        board.set_piece_at(square, chess.Piece(piece, color)) 
        if board.is_valid() or not check_validity:
            break
        else:
            board.remove_piece_at(square)

def generate_random_fen():
    # Generate a random position, where
    # for each side there is a king and optionally a queen and/or a rook and/or a pawn
    
    board = chess.Board()
    board.clear_board()
    board.castling_rights = 0
    board.turn = chess.WHITE if np.random.randint(0, 2) == 0 else chess.BLACK

    # Put kings first because positions without them are illegal
    put_piece(board, chess.WHITE, chess.KING, check_validity=False)
    put_piece(board, chess.BLACK, chess.KING, check_validity=True)

    
    color_piece_count = np.random.choice([
        {chess.WHITE: {chess.QUEEN: 1,chess.PAWN: 0}, chess.BLACK: {chess.QUEEN: 0,chess.PAWN: 0}},
        {chess.WHITE: {chess.QUEEN: 0,chess.PAWN: 1}, chess.BLACK: {chess.QUEEN: 0,chess.PAWN: 0}},
        {chess.WHITE: {chess.QUEEN: 0,chess.PAWN: 0}, chess.BLACK: {chess.QUEEN: 1,chess.PAWN: 0}},
        {chess.WHITE: {chess.QUEEN: 0,chess.PAWN: 0}, chess.BLACK: {chess.QUEEN: 0,chess.PAWN: 1}},
        ])
        

    for color in chess.COLORS:
        for piece, count in color_piece_count[color].items():
            for _ in range(count):
                put_piece(board, color, piece)
    
    return board.fen()
                

N_SAMPLES = 100000

X_rows = []
evals = []
fens = []

for _ in range(N_SAMPLES):
    fen = generate_random_fen()
    fens.append(fen)
    
    board = chess.Board(fen)
    
    X_row = bitarray_to_ndarray(BoardEncoder.encode(board))
    X_rows.append(X_row)

    dtz = tablebase.probe_dtz(board)
    
    evals.append(dtz_to_eval(dtz))

X = np.array(X_rows)
y = np.array(evals)  

In [10]:
X.shape

(100000, 837)

In [11]:
y.shape

(100000,)

## Crossvalidation to see if MLP regressor is a good choice

In [12]:
reg = MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000, verbose=True, alpha=0.5)

cv_score = cross_val_score(reg, X, y, cv=5, scoring='neg_mean_absolute_error')

print(f"Crossvalidation scores: {cv_score}")
print(f"Mean: {cv_score.mean()}")
print(f"Std: {cv_score.std()}")

Iteration 1, loss = 1703.32371643
Iteration 2, loss = 488.80337755
Iteration 3, loss = 445.01355708
Iteration 4, loss = 403.88790431
Iteration 5, loss = 364.41191236
Iteration 6, loss = 328.12483349
Iteration 7, loss = 294.21829638
Iteration 8, loss = 264.11518751
Iteration 9, loss = 239.06953842
Iteration 10, loss = 218.10925252
Iteration 11, loss = 202.72591118
Iteration 12, loss = 187.90225959
Iteration 13, loss = 175.23907353
Iteration 14, loss = 163.68544802
Iteration 15, loss = 154.45758432
Iteration 16, loss = 145.19960749
Iteration 17, loss = 136.67059087
Iteration 18, loss = 129.47362318
Iteration 19, loss = 121.63653178
Iteration 20, loss = 113.95165940
Iteration 21, loss = 107.63724418
Iteration 22, loss = 100.10719278
Iteration 23, loss = 95.15595879
Iteration 24, loss = 89.23338265
Iteration 25, loss = 85.31293287
Iteration 26, loss = 79.75672074
Iteration 27, loss = 76.62622311
Iteration 28, loss = 72.38665520
Iteration 29, loss = 68.51046139
Iteration 30, loss = 65.67529

## Proper training

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
reg.fit(X_train, y_train)
mse = reg.score(X_test, y_test)

print(f"Mean squared error: {mse}")

Iteration 1, loss = 1584.28725256
Iteration 2, loss = 481.95163780
Iteration 3, loss = 435.38136582
Iteration 4, loss = 387.38263376
Iteration 5, loss = 338.71017664
Iteration 6, loss = 298.88588138
Iteration 7, loss = 266.14195645
Iteration 8, loss = 241.53696283
Iteration 9, loss = 220.24892459
Iteration 10, loss = 203.30218328
Iteration 11, loss = 189.25050072
Iteration 12, loss = 179.75293424
Iteration 13, loss = 169.19430729
Iteration 14, loss = 161.05475306
Iteration 15, loss = 152.16622192
Iteration 16, loss = 147.69550470
Iteration 17, loss = 140.86009110
Iteration 18, loss = 134.47376824
Iteration 19, loss = 129.50207721
Iteration 20, loss = 125.66232566
Iteration 21, loss = 120.66184832
Iteration 22, loss = 117.35042268
Iteration 23, loss = 114.06602498
Iteration 24, loss = 111.51779102
Iteration 25, loss = 108.38700606
Iteration 26, loss = 104.55923374
Iteration 27, loss = 102.00537090
Iteration 28, loss = 99.74392155
Iteration 29, loss = 98.25715623
Iteration 30, loss = 95.

## Saving the model

In [14]:
path = f'../models/model-{datetime.now().strftime("%Y%m%d%H%M%S")}.pkl'
os.mknod(path)

with open(path, 'ab') as file:
    pickle.dump(reg, file)

## Loading the model

In [23]:
load_path = '../models/model-20240109150324.pkl'
reg: MLPRegressor
with open(load_path, 'rb') as file:
    reg = pickle.load(file)

## Model predictions

In [24]:
def get_estimation(reg, fen):
    board = chess.Board(fen)
    X = bitarray_to_ndarray(BoardEncoder.encode(board)).reshape(1, -1)
    return reg.predict(X)[0]


White to move:

![](https://fen2image.chessvision.ai/8/8/8/8/3k4/8/3K4/3Q4_w_-_-_0_1?color=white)

In [25]:
get_estimation(reg, '8/8/8/8/3k4/8/3K4/3Q4 w - - 0 1')

91.3298840046165


White to move:

![](https://fen2image.chessvision.ai/k7/p7/8/K7/8/8/8/8_w_-_-_0_1?color=white)

White to move:

![](https://fen2image.chessvision.ai/4k3/8/4K3/4P3/8/8/8/8_w_-_-_0_1?color=white)

In [29]:
get_estimation(reg, '4k3/8/4K3/4P3/8/8/8/8 w - - 0 1')

8.351667841368029

White to move:

![](https://fen2image.chessvision.ai/8/8/4k3/4P3/4K3/8/8/8_w_-_-_0_1?color=white)

In [30]:
get_estimation(reg, '8/8/4k3/4P3/4K3/8/8/8 w - - 0 1')

15.41495722508689

White to move:

![](https://fen2image.chessvision.ai/8/4P3/4K3/8/8/8/8/7k_w_-_-_0_1?color=white)

In [45]:
get_estimation(reg, '8/4P3/4K3/8/8/8/8/7k w - - 0 1')

104.62148691642241

White to move:

![](https://fen2image.chessvision.ai/8/8/8/7K/8/2k5/4P3/8_b_-_-_11_6?color=white)

In [43]:
get_estimation(reg, '8/8/8/7K/8/2k5/4P3/8 w - - 11 6')

102.71839235947378

In [49]:
get_estimation(reg, '8/3Q4/k7/8/8/8/1K6/8 w - - 5 27')


96.70304034337242

In [50]:
with open('../models/model-20231218192512.pkl', 'rb') as file:
    model_KQ_K = pickle.load(file)

In [51]:
get_estimation(model_KQ_K, '8/3Q4/k7/8/8/8/1K6/8 w - - 5 27')

89.24994928207494